In [3]:
pip install earthengine-api

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
import folium
from folium import plugins

# GEE 초기화
ee.Initialize()

# 지정된 영역 정의
coordinates = [
    [126.43355001359191, 35.84994374524949],
    [126.43355001359191, 35.80914407218876],
    [126.50708939726752, 35.80914407218876],
    [126.50708939726752, 35.84994374524949]
]

# ee.Geometry 객체로 변환
geometry = ee.Geometry.Polygon([coordinates])

# Sentinel-2 이미지 컬렉션 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))  # 구름이 적은 이미지 선택

# NDVI 계산 함수 정의
def calcNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# NDVI 적용
withNDVI = sentinel2.map(calcNDVI)

# NDVI의 평균값 계산
meanNDVI = withNDVI.mean().select('NDVI')

# Zonal Statistics
stats = meanNDVI.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=geometry,
    scale=10
)

# 결과 출력
print('Mean NDVI:', stats.getInfo()['NDVI'])

# 맵 생성
map = folium.Map(location=[35.8295439, 126.4704967], zoom_start=12)

# NDVI 레이어 추가
map.addLayer(meanNDVI, {'min': 0, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'Mean NDVI')
# 영역 레이어 추가
folium.GeoJson(geometry.getInfo(), name='geojson').add_to(map)

# 맵 출력
map.add_child(folium.LayerControl())
map

ModuleNotFoundError: No module named 'folium'